In [2]:
import torch
from torch.utils import data
import torch.nn as nn # 신경망 모델
from torch import optim # optimizer 모듈
import torch.nn.functional as F # nn모듈의 함수 버전
from torchvision import datasets, transforms # 패션 MNIST

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
DEVICE

device(type='cpu')

In [5]:
# 이미지를 텐서로 변경
transform = transforms.Compose([transforms.ToTensor()])

In [6]:
trainset = datasets.FashionMNIST(
    root = "./data/",
    train = True,
    download = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root = "./data/",
    train = False,
    download = True,
    transform = transform
)

100%|█████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [00:01<00:00, 22.6MB/s]


Extracting ./data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data/FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 98.9kB/s]


Extracting ./data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data/FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:02<00:00, 1.87MB/s]


Extracting ./data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data/FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<?, ?B/s]

Extracting ./data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST\raw



In [7]:
batch_size = 32

In [8]:
# 데이터 공급 객체 선언
train_loader = data.DataLoader(
    dataset = trainset,
    batch_size = batch_size
)

test_loader = data.DataLoader(
    dataset = testset,
    batch_size = batch_size
)

In [9]:
class Net(nn.Module):
    def __init__(self):
        '''모델 구조 정의'''
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        # 입력 데이터 = x
        # x.shape = torch.Size([배치사이즈, 색, 높이, 너비])
        x = x.view(-1, 28 * 28)

        # x.shape = torch.Size([배치사이즈, 784])
        x = F.relu(self.fc1(x))

        # x.shape = torch.Size([배치사이즈, 앞 레이어의 출력개수])
        x = self.fc2(x)
        return x

In [10]:
# 모델 선언
model = Net().to(DEVICE)

In [11]:
# 옵티마이저 설정
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [12]:
# 학습 함수 정의
def train(model, train_loader, optimizer):
    # 모델을 학습 모드로 전환
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE 로 보냄
        data = data.to(DEVICE)
        target = target.to(DEVICE)

        # 매 이터레이션마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()

        # 실제 모델의 예측값 받아오기
        output = model(data)

        # 정답 데이터와의 cross entropy loss 계산
        # loss는 mini batch의 클래스의 오차 평균값
        loss = F.cross_entropy(output, target)

        # 기울기 계산
        loss.backward()

        # 가중치 수정
        optimizer.step()

In [16]:
def evaluate(model, test_loader):
    # 모델을 평가모드로 전환
    model.eval()

    # 필요한 변수 초기화
    # test 과정에서의 loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0

    with torch.no_grad(): # 평가 시에는 기울기를 계산하지 않으므로, no_grad 를 명시
        for data, target in test_loader:
            data = data.to(DEVICE)
            target = target.to(DEVICE)

            output = model(data)

            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target, reduction = "sum").item()

            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 예측과 정답을 비교하여 일치하면 correct 에 1 더하기
            pred = output.max(1, keepdim = True)[1]

            # eq() : 값이 일치하면 1, 아니면 0
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # 정확도 계산
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [17]:
for epoch in range(1, 6):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f"[{epoch}] Test Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.2f}%")

[1] Test Loss : 0.5415, Accuracy : 81.48%
[2] Test Loss : 0.5051, Accuracy : 82.32%
[3] Test Loss : 0.4844, Accuracy : 82.93%
[4] Test Loss : 0.4701, Accuracy : 83.41%
[5] Test Loss : 0.4589, Accuracy : 83.70%
